In [1]:
# handling BAG
import h5py
from io import BytesIO, StringIO

from lxml import etree

import xml.etree.ElementTree as ET

import numpy as np

import rasterio


In [2]:
def getbagcrs(metadata):
    #with help from Alex Leith
    
    root = ET.fromstring(metadata)
    #root = tree.getroot()

    namespaces = {
        'gmd': "http://www.isotc211.org/2005/gmd",
        'gco': "http://www.isotc211.org/2005/gco"
    }

    referencesystem = root.find('gmd:referenceSystemInfo', namespaces)[0]

    referencesystemid = referencesystem.find('gmd:referenceSystemIdentifier', namespaces).find('gmd:RS_Identifier', namespaces)

    crswkt = referencesystemid.find('gmd:code', namespaces).find('gco:CharacterString', namespaces).text

    return crswkt

In [3]:
bag = h5py.File('../../bag-samples/grid0_ellipsoid_1m.bag')

In [4]:
bag

<HDF5 file "grid0_ellipsoid_1m.bag" (mode r+)>

In [5]:
root = bag['BAG_root']

In [6]:
metadata = root['metadata']

In [7]:
metadata.value

array([b'<', b'?', b'x', ..., b'>', b'\n', b''], dtype='|S1')

In [8]:
buffer = BytesIO(metadata.value[:-1])

In [9]:
buffer.getvalue()

b'<?xml version="1.0" encoding="UTF-8"?>\n<gmi:MI_Metadata xmlns:gmi="http://www.isotc211.org/2005/gmi" xmlns:gmd="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:gml="http://www.opengis.net/gml/3.2" xmlns:gco="http://www.isotc211.org/2005/gco" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:bag="http://www.opennavsurf.org/schema/bag"><gmd:fileIdentifier><gco:CharacterString>86e1f452-89d0-4db0-a5a7-da6312db412e</gco:CharacterString></gmd:fileIdentifier><gmd:language><gmd:LanguageCode codeList="http://www.loc.gov/standards/iso639-2/" codeListValue="eng">eng</gmd:LanguageCode></gmd:language><gmd:characterSet><gmd:MD_CharacterSetCode codeList="http://www.isotc211.org/2005/resources/Codelist/gmxCodelists.xml#MD_CharacterSetCode" codeListValue="utf8">utf8</gmd:MD_CharacterSetCode></gmd:characterSet><gmd:hierarchyLevel><gmd:MD_ScopeCode codeList="http://www.isotc211.org/2005/resources/Codelist/gmxCodelists.xml#MD_ScopeCode" codeListValue="datase

In [14]:
this = getbagcrs(buffer.getvalue().decode())

In [15]:
bag.close()

In [16]:
this

'PROJCS["EPSG:32755",GEOGCS["WGS_1984",DATUM["WGS_1984",SPHEROID["WG84",6378137,298.2572235604902,AUTHORITY["","WG84"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree (supplier to define representation)",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",147],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",10000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'

In [17]:
epsgcode = rasterio.crs.CRS.to_epsg(rasterio.crs.CRS.from_string(this))

In [18]:
epsgcode

32755